In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *


In [4]:
sc = SparkContext()
ssc =  SQLContext(sc)

In [6]:
schema =  StructType().add('Year',IntegerType(),True).add('Quarter',IntegerType(),True).add('rev_per_seat',DoubleType(),True).add('num_of_seats',IntegerType(),True)

In [8]:
df1 = ssc.read.format('csv').option('header','True').schema(schema).load('hdfs://nameservice1/user/bigcdac432589/EXAM/airlines.csv')

In [10]:
df1.show(5)

+----+-------+------------+------------+
|Year|Quarter|rev_per_seat|num_of_seats|
+----+-------+------------+------------+
|1995|      1|       296.9|       46561|
|1995|      2|       296.8|       37443|
|1995|      3|      287.51|       34128|
|1995|      4|      287.78|       30388|
|1996|      1|      283.97|       47808|
+----+-------+------------+------------+
only showing top 5 rows



In [11]:
df1.registerTempTable('airlines')

In [26]:
df1.repartition(1)

DataFrame[Year: int, Quarter: int, rev_per_seat: double, num_of_seats: int]

In [27]:
que1 = ssc.sql('SELECT Year, sum(num_of_seats) as total_passengers FROM airlines GROUP BY Year ORDER BY total_passengers DESC')

In [16]:
que1.first()

Row(Year=2007, total_passengers=176299)

In [28]:
que1.show()

+----+----------------+
|Year|total_passengers|
+----+----------------+
|2007|          176299|
|2013|          173676|
|2001|          173598|
|1996|          167223|
|2008|          166897|
|2012|          166076|
|2015|          165438|
|2004|          164800|
|2010|          163741|
|2014|          159823|
|1997|          157972|
|2003|          156153|
|2000|          154376|
|2006|          153789|
|2002|          152195|
|2005|          150610|
|2009|          150308|
|1999|          150000|
|1995|          148520|
|2011|          142647|
+----+----------------+
only showing top 20 rows



In [19]:
que2 = ssc.sql('SELECT Year, round((sum(rev_per_seat * num_of_seats)/1000000),2) as total_revenue_Millions FROM airlines GROUP BY Year ORDER BY total_revenue_Millions DESC')

In [29]:
que2.show()

+----+----------------------+
|Year|total_revenue_Millions|
+----+----------------------+
|2013|                 66.36|
|2014|                 62.62|
|2015|                 62.38|
|2012|                  62.2|
|2008|                 57.65|
|2007|                 57.31|
|2001|                 55.53|
|2010|                 54.86|
|2000|                 52.34|
|2011|                 51.89|
|2004|                 50.63|
|2006|                 50.44|
|2003|                 49.27|
|1999|                 48.76|
|2002|                  47.5|
|2009|                 46.75|
|2005|                 46.38|
|1996|                 46.36|
|1997|                 45.39|
|1995|                 43.49|
+----+----------------------+
only showing top 20 rows



In [21]:
que2.first()

Row(Year=2013, total_revenue_Millions=66.36)

In [22]:
que3 = ssc.sql('SELECT Year,Quarter, round((sum(rev_per_seat * num_of_seats)/1000000),2) as total_revenue_Millions FROM airlines GROUP BY Year,Quarter ORDER BY total_revenue_Millions DESC')

In [30]:
que3.show()

+----+-------+----------------------+
|Year|Quarter|total_revenue_Millions|
+----+-------+----------------------+
|2014|      4|                 18.82|
|2013|      1|                 18.57|
|2013|      3|                 18.18|
|2015|      2|                 17.32|
|2000|      1|                 16.39|
|2010|      1|                  16.3|
|2012|      4|                 16.09|
|2014|      3|                 15.96|
|2012|      3|                 15.95|
|1999|      1|                 15.74|
|2004|      1|                  15.7|
|2008|      1|                 15.63|
|2007|      2|                 15.54|
|2015|      4|                 15.49|
|2008|      4|                 15.46|
|2012|      2|                 15.45|
|2006|      3|                 15.34|
|2001|      1|                 15.25|
|2010|      4|                 15.16|
|2011|      3|                 15.12|
+----+-------+----------------------+
only showing top 20 rows



In [24]:
que3.first()

Row(Year=2014, Quarter=4, total_revenue_Millions=18.82)

## Using RDD

In [31]:
rdd1 = sc.textFile('hdfs://nameservice1/user/bigcdac432589/EXAM/airlines.csv')

In [32]:
header = rdd1.first()

In [33]:
print(header)

Year,Quarter,Average revenue per seat,total no. of booked seats


In [34]:
rdd2 = rdd1.filter(lambda s:s!=header)

In [42]:
que1rdd1 = rdd2.map(lambda s:(s.split(',')[0],int(s.split(',')[3])))

In [43]:
que1rdd2 = que1rdd1.reduceByKey(lambda a,b:a+b)

In [44]:
que1rdd3=que1rdd2.sortBy(lambda s:-s[1])

In [47]:
que1rdd3.first()

('2007', 176299)